Import packages

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, plot, iplot
import sqlite3
import re
import Code.Preparation as prep
import itertools
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
warnings.filterwarnings('ignore')

### Clean and combine happiness reports dataset

Load datasets

In [2]:
df2021 = pd.read_csv(prep.path('happy/2021.csv'))
df2021['Year'] = 2021

In [3]:
dfhappy = pd.read_csv(prep.path('happy/world-happiness-report.csv'))
RateDf = pd.read_csv(prep.path('sucide.csv'))
RateDf = RateDf.rename(columns={'country': 'Country'})

Change names to match corresponding columns and merge dataframes

In [4]:
df2021 = df2021[['Country name','Year', 'Ladder score','Social support','Healthy life expectancy','Logged GDP per capita','Freedom to make life choices','Generosity','Perceptions of corruption']]
df2021 = df2021.rename(columns={'Year': 'year', 'Ladder score': 'Life Ladder', 'Healthy life expectancy': 'Healthy life expectancy at birth', 'Logged GDP per capita': 'Log GDP per capita', })
dfhappy = dfhappy.merge(df2021, how='outer').drop(columns=['Positive affect', 'Negative affect'])
dfhappy = dfhappy.rename(columns={'Country name': 'Country'})

Check for null values

In [5]:
dfhappy.isna().sum()

Country                               0
year                                  0
Life Ladder                           0
Log GDP per capita                   36
Social support                       13
Healthy life expectancy at birth     55
Freedom to make life choices         32
Generosity                           89
Perceptions of corruption           110
dtype: int64

Interpolate null values

In [6]:
dfhappy = dfhappy.interpolate()

Save data to csv for modeling. 

In [7]:
# Export dataset to csv
dfhappy.to_csv('Data/CleanedHappy.csv')

### World Dataset

In [8]:
datadf = pd.read_csv(prep.path('WDI/WDIData.csv'))
countrydf = pd.read_csv(prep.path('WDI/WDICountry.csv'))
df = pd.read_csv(prep.path('World_Development/Indicators.csv'))

In [9]:
Indicator_array =  df[['IndicatorName','IndicatorCode']].drop_duplicates().values

Search tool for relevent features that could have a impact on suicides. 

In [10]:
modified_indicators = []
unique_indicator_codes = []
for ele in Indicator_array:
    indicator = ele[0]
    indicator_code = ele[1].strip()
    if indicator_code not in unique_indicator_codes:
        # delete , ( ) from the IndicatorNames
        new_indicator = re.sub('[,()]',"",indicator).lower()
        # replace - with "to" and make all words into lower case
        new_indicator = re.sub('-'," to ",new_indicator).lower()
        modified_indicators.append([new_indicator,indicator_code])
        unique_indicator_codes.append(indicator_code)

Indicators = pd.DataFrame(modified_indicators,columns=['IndicatorName','IndicatorCode'])
Indicators = Indicators.drop_duplicates()

In [11]:
key_word_dict = {}
key_word_dict['Food'] = ['food','grain','nutrition','calories']
key_word_dict['Foreign'] = ['foreign']
key_word_dict['Health'] = ['health','desease','hospital','mortality','doctor', 'mental']
key_word_dict['Economy'] = ['income','gdp','gni','deficit','budget','market','stock','bond','infrastructure', 'investment']
key_word_dict['Education'] = ['education','literacy', 'school', 'college']
key_word_dict['Energy'] = ['fuel','energy','power','emission','electric','electricity', 'water']
key_word_dict['Employment'] =['employed','employment','umemployed','unemployment']
key_word_dict['Rural'] = ['rural','village']
key_word_dict['Urban'] = ['urban','city']
key_word_dict['Social Programs'] = ['social', 'welfare']
key_word_dict['Tech'] = ['technology', 'tech', 'phone', 'mobile', 'broadband', 'cable', 'telephone']
key_word_dict['Trade'] = ['trade','import','export','good','shipping','shipment']
key_word_dict['Water'] = ['water', 'sanitation', 'sanitary']
key_word_dict['Access'] = ['access']

In [12]:
feature = 'Social Programs'
for indicator in Indicators.values:
    for w in key_word_dict[feature]:
        word_list = indicator[0].split()
        if w in word_list or w+'s' in word_list:
            # Uncomment this line to print the indicators explicitely
            print(indicator)
            break

['social contributions % of revenue' 'GC.REV.SOCL.ZS']
['social contributions current lcu' 'GC.REV.SOCL.CN']
['adequacy of social insurance programs % of total welfare of beneficiary households'
 'per_si_allsi.adq_pop_tot']
['adequacy of social protection and labor programs % of total welfare of beneficiary households'
 'per_allsp.adq_pop_tot']
['adequacy of unemployment benefits and almp % of total welfare of beneficiary households'
 'per_lm_alllm.adq_pop_tot']
['benefits incidence in poorest quintile %  to  all social insurance'
 'per_si_allsi.ben_q1_tot']
['benefits incidence in poorest quintile %  to all social protection and labor'
 'per_allsp.ben_q1_tot']
['coverage %  to  all social insurance' 'per_si_allsi.cov_pop_tot']
['coverage %  to all social protection and labor' 'per_allsp.cov_pop_tot']
['coverage %  to  all social assistance' 'per_sa_allsa.cov_pop_tot']
['adequacy of social safety net programs % of total welfare of beneficiary households'
 'per_sa_allsa.adq_pop_tot']
['

In [13]:
worlddf = pd.DataFrame(datadf.groupby(['Country Name','Indicator Name']).mean().stack())
worlddf = worlddf.reset_index()
worlddf = worlddf.rename(columns={0: 'Value', 'level_2': 'year', 'Country Name': 'Country', 'Indicator Name': 'IndicatorName'})
worlddf = worlddf[worlddf.year >= '1985']

In [23]:
worlddf

,Country,IndicatorName,year,Value
0,Afghanistan,"2005 PPP conversion factor, GDP (LCU per inter...",2005,15.13216
1,Afghanistan,"2005 PPP conversion factor, private consumptio...",2005,16.70967
2,Afghanistan,ARI treatment (% of children under 5 taken to ...,2011,60.50000
3,Afghanistan,ARI treatment (% of children under 5 taken to ...,2015,61.50000
4,Afghanistan,Access to clean fuels and technologies for coo...,2000,8.80000
...,...,...,...,...
8263053,Zimbabwe,Women's share of population ages 15+ living wi...,2013,58.80000
8263054,Zimbabwe,Women's share of population ages 15+ living wi...,2014,58.90000
8263055,Zimbabwe,Women's share of population ages 15+ living wi...,2015,58.90000
8263056,Zimbabwe,Women's share of population ages 15+ living wi...,2016,59.00000


In [24]:
Clean_world = worlddf[(worlddf.IndicatorName == 'Multilateral debt (% of total external debt)')| 
        (worlddf.IndicatorName == 'Urban population growth (annual %)')|
        (worlddf.IndicatorName == 'Rural population growth (annual %)')|
        (worlddf.IndicatorName == 'Mobile cellular subscriptions (per 100 people)')|
        (worlddf.IndicatorName == 'Fixed broadband subscriptions (per 100 people)')|
        (worlddf.IndicatorName == 'Foreign direct investment, net inflows (% of GDP)')|
        (worlddf.IndicatorName == 'GNI per capita, Atlas method (current US$)')|
        (worlddf.IndicatorName == 'Primary education, duration (years)')|
        (worlddf.IndicatorName == 'Primary education, duration (years)')|
        (worlddf.IndicatorName == 'Inflation, consumer prices (annual %)')|
        (worlddf.IndicatorName == 'Adequacy of social safety net programs (% of total welfare of beneficiary households)')|
        (worlddf.IndicatorName == 'Access to electricity (% of population)')|
        (worlddf.IndicatorName == 'People using at least basic drinking water services (% of population)')]


In [25]:
Clean_world.to_csv('Data/WorldCleanedDf.csv')


We start off by creating categorical variables for our suicide dataset and ordering our data frames by country and year in the index. We then drop all the years in our categorical dataset and only keep the most recent year to combine the categorical data frame with our continuous variables data frame which in turn gives us one clean processed data frame

***
### Model Preprocessing


We start off by creating categorical variables for our suicide dataset and ordering our data frames by country and year in the index. We then drop all the years in our categorical dataset and only keep the most recent year to combine the categorical data frame with our continuous variables data frame which in turn gives us one clean processed data frame

In [26]:
# Make copy wiht only categorical variables
RateCat = RateDf.copy().set_index(['Country', 'year'])[['sex', 'age', 'generation']]

# Initiate label encoder and apply to our categorical columns.
le = LabelEncoder()
RateCat['sex'] = le.fit_transform(RateCat['sex'])
RateCat['age'] = le.fit_transform(RateCat['age'])
RateCat['generation'] = le.fit_transform(RateCat['generation'])
RateCat.reset_index(inplace=True)

# Drop all but keep the last categorical data for each year(each year has 12 different categories for data so only keep the last for each)
RateCat.drop_duplicates(subset=['Country', 'year'], keep='last', inplace=True)

#Create dataframe with continuous variables and drop population as it's not relevent and could cause multicollenuarity issues. 
RateCon = RateDf.copy().groupby(['Country', 'year']).mean().drop(columns=['population', 'HDI for year'])


In [27]:
# Merge the two dataframes on country and year as our index. 
RateCon = RateCon.merge(RateCat, how='outer', on=['Country', 'year']).groupby(['Country', 'year']).mean().astype(float)

Our world dataset does not have a target variable, only features to use as predictors for either suicide rates or happiness scores. I've manually gone through over a thousand features and tried to find only relevant features to our business problem and didn't have too many missing values.

We iterate through the dataset and create a temp data frame out of a feature then combine them all into one under the same index country and year while printing a preview of the feature mean for the world. 

In [28]:
features = ['Urban population growth (annual %)', 'Rural population growth (annual %)', 'Mobile cellular subscriptions (per 100 people)',
            'Fixed broadband subscriptions (per 100 people)', 'Foreign direct investment, net inflows (% of GDP)', 'GNI per capita, Atlas method (current US$)', 'Primary education, duration (years)', 
           'Inflation, consumer prices (annual %)', 'Access to electricity (% of population)',
           'People using at least basic drinking water services (% of population)']

def get_world_df(features):
    
    listi = []
    
    for name in features:
        name = Clean_world[Clean_world.IndicatorName == name].drop(columns='IndicatorName').groupby(['Country', 'year']).mean().rename(columns={'Value': name})
        print(name.mean())
        listi.append(name)
    return listi
world = get_world_df(features)

World_Over = pd.concat(world).groupby(['Country', 'year']).mean()

Urban population growth (annual %)    2.37697
dtype: float64
Rural population growth (annual %)    0.480022
dtype: float64
Mobile cellular subscriptions (per 100 people)    38.65041
dtype: float64
Fixed broadband subscriptions (per 100 people)    8.618271
dtype: float64
Foreign direct investment, net inflows (% of GDP)    4.055229
dtype: float64
GNI per capita, Atlas method (current US$)    8883.122807
dtype: float64
Primary education, duration (years)    5.699107
dtype: float64
Inflation, consumer prices (annual %)    25.482019
dtype: float64
Access to electricity (% of population)    75.036003
dtype: float64
People using at least basic drinking water services (% of population)    84.189864
dtype: float64


***
Our last dataset with happiness features and our happiness score target('Life Ladder'). We set the same index for our data to be able to merge in the right locations then merge it with our suicide dataset. Finally we merge that dataset with the world features dataset to create one cohesive data frame dropping any values that are missing. Doing this will remove some data and/or years but we will be left with only valid data from all 3 data frames. 

In [29]:
RateCon.reset_index(inplace=True)

In [30]:
RateCon.year = RateCon.year.astype(str)
dfhappy.year = dfhappy.year.astype(str)

In [31]:
dfhappy = dfhappy.groupby(['Country', 'year']).mean()

In [32]:
HappyRate = dfhappy.merge(RateCon, on=['Country', 'year']).groupby(['Country', 'year']).mean().dropna()

In [33]:
AllThree = World_Over.merge(HappyRate, on=['Country', 'year']).groupby(['Country', 'year']).mean().dropna() #.reset_index()

In [34]:
AllThree.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 519 entries, ('Albania', '2007') to ('Uruguay', '2015')
Data columns (total 23 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Urban population growth (annual %)                                     519 non-null    float64
 1   Rural population growth (annual %)                                     519 non-null    float64
 2   Mobile cellular subscriptions (per 100 people)                         519 non-null    float64
 3   Fixed broadband subscriptions (per 100 people)                         519 non-null    float64
 4   Foreign direct investment, net inflows (% of GDP)                      519 non-null    float64
 5   GNI per capita, Atlas method (current US$)                             519 non-null    float64
 6   Primary education, duration (years)                     

In [35]:
AllThree.to_csv('Data/Combined_df.csv')